### prerequisites:
- get precompiled opencv training binaries from https://sourceforge.net/projects/opencvlibrary/files/3.4.16/opencv-3.4.16-vc14_vc15.exe/download
    - in extracted folder \opencv\build\x64\vc15\bin
        - opencv_createsamples.exe
        - opencv_traincascade.exe
- positive images, in annotation txt
- negative images

### docs & info:
- https://docs.opencv.org/4.10.0/dc/d88/tutorial_traincascade.html
- https://www.youtube.com/watch?v=XrCAvs9AePM


In [2]:
# generate the file with negative images
import os
with open('neg.txt', 'w') as f:
    # loop over all the filenames
    for filename in os.listdir('negative/croccy'):
        f.write('negative/croccy/' + filename + '\n')

In [2]:
# generate the file with positive samples
from subprocess import Popen, PIPE, STDOUT
from IPython.core.magic import register_line_magic
@register_line_magic
def runrealcmd(command):
    process = Popen(command, stdout=PIPE, shell=True, stderr=STDOUT, bufsize=1, close_fds=True)
    for line in iter(process.stdout.readline, b''):
        print(line.rstrip().decode('utf-8'))
    process.stdout.close()
    process.wait()

runrealcmd("bins\opencv_createsamples.exe -vec positive_samples -info anno.txt -num 160 -show -w 24 -h 24")

C:\Program Files\Python311\Lib\subprocess.py:1014: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)


Info file name: anno.txt
Img file name: (NULL)
Vec file name: positive_samples
BG  file name: (NULL)
Num: 160
BG color: 0
BG threshold: 80
Invert: FALSE
Max intensity deviation: 40
Max x angle: 1.1
Max y angle: 1.1
Max z angle: 0.5
Show samples: TRUE
Scale: 4
Width: 24
Height: 24
Max Scale: -1
RNG Seed: 12345
Create training samples from images collection...
Done. Created 160 samples


In [3]:
# do the actual training

runrealcmd("bins\opencv_traincascade.exe -data cascade/ -vec positive_samples -bg neg.txt -numPos 160 -numNeg 160 -numStages 10 -w 24 -h 24 -precalcValBufSize 1024 -precalcIdxBufSize 1024 -featureType HAAR -minHitRate 0.999 -maxFalseAlarmRate 0.5 -mode ALL")

---------------------------------------------------------------------------------
Training parameters are pre-loaded from the parameter file in data folder!
Please empty this folder if you want to use a NEW set of training parameters.
---------------------------------------------------------------------------------
PARAMETERS:
cascadeDirName: cascade/
vecFileName: positive_samples
bgFileName: neg.txt
numPos: 160
numNeg: 160
numStages: 10
precalcValBufSize[Mb] : 1024
precalcIdxBufSize[Mb] : 1024
acceptanceRatioBreakValue : -1
stageType: BOOST
featureType: HAAR
sampleWidth: 24
sampleHeight: 24
boostType: GAB
minHitRate: 0.995
maxFalseAlarmRate: 0.5
weightTrimRate: 0.95
maxDepth: 1
maxWeakCount: 100
mode: BASIC
Number of unique features given windowSize [24,24] : 162336

Stages 0-9 are loaded
